<center> <h1> LBSN Project </h1> </center>


Authors: Cédric Allain, Clotilde Miura, Manon Rivoire, Adriano Del Gallo, Saousan Kaddami

Group #4

April 11th, 2020

The purpose of this lab is to implement a location-based recommender system from regular location-based social network (LBSN ) data. Usually LBSN data give precise information regarding the locations visited by the users. These data also reveal who are the friends of the users. These two information (geographical + social) can be exploited into a standard collaborative filtering approach.

We build in this notebook blocks of the model developped by Zhang and Chow, 2013, iGSLR: personalized geo-social location recommendation: a kernel density estimation approach.

Source: https://dl.acm.org/doi/10.1145/2525314.2525339

In [93]:
use_colab = False

If you have a look here :  https://jupyter-notebook.readthedocs.io/en/stable/config.html You will see that the limit is 500Mo "NotebookApp.max_buffer_sizeInt"

In [94]:
"jupyter notebook --NotebookApp.max_buffer_size=<64>"

'jupyter notebook --NotebookApp.max_buffer_size=<64>'

In [95]:
!pip install surprise

In [128]:
import os
import sys
import re
import random
import pandas as pd
pd.set_option('display.max_columns', 100)
pd.set_option("display.max_rows", 100)
import numpy as np 
import pickle
import math 
from collections import defaultdict 
from tqdm import tqdm

from surprise import AlgoBase
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split, KFold

import matplotlib.pyplot as plt 
import seaborn as sns 
sns.set(color_codes=True)

## 1. Load the data

The input dataset for this lab was collected from Gowalla, a popular LBSN that is often used to test recommendation methods with geographical dimensions. In practice the dataset contains user profiles, user friendship, location profiles, and users’ check-in history made before June 1, 2011. It contains 36,001,959 check-ins made by 407,533 users over 2,724,891 locations.

Data can be downloaded here: https://snap.stanford.edu/data/loc-gowalla.html

In [97]:
if use_colab:
    from google.colab import drive
    drive.mount("/content/drive", force_remount=True)
    %cd drive/'My Drive'/recommender_systems_2020/Data/gowalla
else:
    %cd './gowalla'

[Errno 2] No such file or directory: './gowalla'
/Users/cedricallain/Documents/ENSAE/3A/S2/Systèmes de recommandation/Projet/gowalla


In [98]:
# Load data into DataFrames

checkins_df = pd.read_csv('gowalla_checkins.csv')
friendship_df = pd.read_csv('gowalla_friendship.csv')
locations_df = pd.read_csv('gowalla_locations.csv')
users_df = pd.read_csv('gowalla_userinfo.csv')

## 2. Processing of the Dataset

- Surprise library cannot handle implicit feedback 
- sigmoidal transformation : convert a frequency into a specific range 
- goal is to generate recommendations based on the building blocks ==> surprise frameworks 

In [99]:
dict_df = {'checkins_df': checkins_df,
           'friendship_df': friendship_df,
           'locations_df': locations_df,
           'users_d': users_df}

for name, df in dict_df.items():
    print(name, ', shape:', df.shape)
    print(df.head(), '\n')

checkins_df , shape: (36001959, 2)
   userid  placeid
0    1338   482954
1    1338   580963
2    1338   365256
3    1338    89504
4    1338  1267135 

friendship_df , shape: (4418339, 2)
   userid1  userid2
0        1    63488
1        1        2
2        1        3
3        1        4
4        1        5 

locations_df , shape: (2724891, 5)
     id         lng        lat  checkins_count  users_count
0  8904  -94.607499  39.052318             114           21
1  8932  -97.254356  32.927662              67           48
2  8936  -94.591995  39.053318              75           46
3  8938  -94.590311  39.052824             438           94
4  8947 -122.029631  37.331880            3100         1186 

users_d , shape: (407533, 3)
   id  friends_count  checkin_num
0   1            372         1766
1   2            775         2892
2   3            100         3021
3   4            179         1325
4   5            525         3215 



### Filtering out some users and POIs




In [100]:
users_df.checkin_num.max()

46981

Problem:
- This user made more than 46981 different checkins.
- It means we would have to compute $\begin{pmatrix}46981 \\
                                                        2\end{pmatrix}$ distances between the different pairs of locations this user visited, which is equal to more than 1 billion...

Solution:
- Filter out users who have less than 15 checkins and more than 100 checkins
- Filter out POIs/locations which have less than 30 visitors

Remark: a similar filter is done in Guo et al., 2019, Network Embedding-Aware Point-of-Interest Recommendation in Location-Based Social Networks

In [101]:
# filter out users who have less than 15 checkins  and more  than 100 checkins
users_df = users_df[(users_df.checkin_num >= 15) & (users_df.checkin_num <=100)]
users_list = users_df.id.tolist()

# filter out POIs/locations which have less than 30 visitors
locations_df = locations_df[locations_df.users_count >= 30]
locations_list = locations_df.id.tolist()

done = False
while not done:
    done = True
    
    old_users_list = users_list.copy()
    old_locations_list = locations_list.copy()
    
    # filter friendship_df to only get friends of our n users
    friendship_df = friendship_df[friendship_df['userid1'].isin(users_list)]
    # but those friends must have more than 15 checkins and less than 100, i.e. be in our list of users
    friendship_df = friendship_df[friendship_df['userid2'].isin(users_list)]
    # recompute users list
    users_list = friendship_df['userid1'].unique().tolist()
    
    # also filter the checkins dataframe
    checkins_df = checkins_df[(checkins_df.userid.isin(users_list)) & (checkins_df.placeid.isin(locations_list))]
    
    # recompute users and locations list
    users_list = checkins_df['userid'].unique().tolist()
    locations_list = checkins_df['placeid'].unique().tolist()
    
    if (len(old_users_list)>len(users_list)) or (len(old_locations_list)>len(locations_list)):
        done = False
        
# filter again users and locations data frames
users_df = users_df[users_df.id.isin(users_list)]
locations_df = locations_df[locations_df.id.isin(locations_list)]
    
print('number of users:', len(users_list))
print('number of locations:', len(locations_list))

number of users: 82410
number of locations: 93680


NB: people who only had friends who made less than 15 checkins ore more than 100 checkins are discarded, even if they made more than 15 checkins or less than 100 checkins themselves.

Thus, restricting checkins_df to the remaining users in friendship_df and locations in the list of locations can discard some users, that is why we do a while loop (it only loops 2 times).

### Build df_user_friends 

For each user, associate the list of his friends and put the result in a new dataframe df_user_friends.

In [102]:
print('shape friendship_df:', friendship_df.shape)
friendship_df.head()

shape friendship_df: (401340, 2)


,userid1,userid2
2886,20,28448
2889,20,9285
2890,20,338695
2898,2097174,334541
2899,2097174,2110997


In [103]:
%%time

df_user_friends = friendship_df.groupby('userid1').agg({'userid2': 'unique'})
df_user_friends.reset_index(inplace=True)
df_user_friends.columns = ['userid', 'list_friends']

print('shape df_user_friends:', df_user_friends.shape)
df_user_friends.head()

shape df_user_friends: (82410, 2)
CPU times: user 27.9 s, sys: 567 ms, total: 28.4 s
Wall time: 30.9 s


,userid,list_friends
0,20,"[28448, 9285, 338695]"
1,76,"[16643, 30855, 98447, 10014, 284839, 136274, 2..."
2,81,"[69506, 165411, 46163]"
3,86,"[2372512, 276655, 2373417]"
4,93,[108309]


### Build df_user_locations

Associate for each user the list of the successive locations he visited and put the result in a new dataframe df_user_locations.

Remark: this dataframe is not used for the creation of df_frequencies

In [104]:
%%time

df_user_locations = checkins_df.groupby('userid').agg({'placeid': 'unique'}) 
df_user_locations.columns = ['list_locations']

print('shape df_user_locations:', df_user_locations.shape)
df_user_locations.head()

shape df_user_locations: (82410, 1)
CPU times: user 23.2 s, sys: 287 ms, total: 23.5 s
Wall time: 23.9 s


,list_locations
userid,
20,"[69875, 1186843, 67957, 68209, 37322, 99630, 4..."
76,"[21076, 107498, 81300, 29778, 43297, 367123, 1..."
81,"[85577, 128873, 6708957, 759363, 7040412, 9226..."
86,"[74468, 36326, 1333629, 38713, 6741441, 69870,..."
93,"[17831, 20918, 9220, 15245, 180328, 381380, 18..."


### Build df_frequencies

Compute for each pair (user, location) its corresponding visit frequency in order to build a new df_frequencies dataframe.

In [105]:
%%time

df_frequencies = checkins_df.groupby(['userid', 'placeid']).agg({'placeid': 'count'})
df_frequencies.columns = ['freq']
df_frequencies.reset_index(inplace=True)

print('shape df_frequencies:', df_frequencies.shape)
df_frequencies.head()

shape df_frequencies: (1084596, 3)
CPU times: user 609 ms, sys: 68.5 ms, total: 677 ms
Wall time: 549 ms


,userid,placeid,freq
0,20,9540,1
1,20,9724,1
2,20,9780,1
3,20,9782,1
4,20,9783,1


Transform and update the frequencies from df_frequencies into the range [0, 10] with the following normalization transformation:

$$x \mapsto 10 \cdot \tanh \left[10 \cdot \frac{x-f_{\min }}{f_{\max }-f_{\min }}\right]$$

where $f_{min}$ and $f_{max}$ refers respectively to the minimum and maximum visit frequencies of the dataset.

In [106]:
def normalize(x, fmin, fmax):
    """Normalization tranformation function applied on an integer
    
    Apply the following normalization function:
    
        10 * tanh(10 * (x-fmin)/(fmax-fmin))
    
    Parameters
    ----------
    x: integer
        The number we want to normalize.
    
    fmin: integer
        The minimum visit frequencies of the dataset.
    
    fmax: integer
        The maximum visit frequencies of the dataset.
    
    Returns
    -------
    res: integer
        input x normalized between 0 and 10
    
    """
    res = 10 * np.tanh(10 * (x-fmin)/(fmax-fmin+1e-16))
    return res


def normalize_serie(serie):
    """Normalization tranformation function applied on a serie of integers
    
    Parameters
    ----------
    serie: pandas Series of integers
        The serie we want to normalize
        
    Returns
    -------
    serie_normalized: pandas Series of integers
        input serie normalized to contained only integers between 0 and 10
    
    """
    fmin = serie.min()
    fmax = serie.max()
    
    serie_normalized = serie.apply(normalize, args=(fmin,fmax))
    
    return serie_normalized

In [107]:
%%time

df_frequencies.freq = normalize_serie(df_frequencies.freq)
df_frequencies.head()

CPU times: user 3.12 s, sys: 78.5 ms, total: 3.2 s
Wall time: 3.24 s


,userid,placeid,freq
0,20,9540,0.0
1,20,9724,0.0
2,20,9780,0.0
3,20,9782,0.0
4,20,9783,0.0


In [108]:
df_frequencies.freq.describe(percentiles=[0.1, 0.5, 0.75, 0.9, 0.95])

count    1.084596e+06
mean     3.781687e-01
std      1.126331e+00
min      0.000000e+00
10%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
90%      1.243530e+00
95%      2.449187e+00
max      1.000000e+01
Name: freq, dtype: float64

Remark: the normalization sets to 0 most of the observations

## 3. Geographical Computations

Compute for each user $u$ the distances $d_{ij}$ between each pair of locations he has been located to.

$$\forall l_i, l_j \in L_u, \quad d_{ij} = \text{distance}(l_i, l_j) $$

where $L_u$ is, for an user $u$, the list of all locations he has been located to.

To compute the distance between two spatial points, i.e. defined by their their latitude and longitude, we use the haversine distance: https://en.wikipedia.org/wiki/Haversine_formula

The package haversine allow us to compute such a distance: https://github.com/mapado/haversine

In [17]:
!pip install haversine

In [18]:
def kernel(x):
    """Kernel function"""
    k = np.exp(-(x**2)/2) / (np.sqrt(2*np.pi))
    return k

In [19]:
def density(new_d, locations_dist, n):
    """Compute the density of any new given distance
    
    Parameters
    ----------
    new_d: integer
        the new distance for which we want to compute the density
        
    locations_dist: list of integers
        distances between pairs of locations
        
    n: double
        number of locations visited by the user
        
    Returns
    -------
    f_hat: integer
        the density computed
    
    """

    sigma_hat = np.std(locations_dist)
    
    h = 1.06 * sigma_hat * n**-1/5
    
    # compute density
    num = np.sum([kernel((new_d - d)/h) for d in locations_dist])
    den = len(locations_dist)*h
    
    if den==0:
        f_hat = 0
    else:
        f_hat = num/den
    
    return f_hat

In [20]:
from itertools import combinations
from haversine import haversine

def geo_proba(user, new_loc, df_user_locations=df_user_locations, unit='km'): 
    """Compute the geographical likelihood that a user will visit a new location
    
    Parameters
    ----------
    user: double
        id of user

    new_loc: integer ou tuple
        if integer, id of the new location
        if tuple, tuple (lat, lng) of the new location
        
    df_user_locations: pandas DataFrame
        dataframe that associates for each user the list of the successive locations he visited
        index: userid
        columns = [list_locations]
        
    unit: string
        the initials of its corresponding unit of measurement
        default is 'km'
        (kilometers = 'km', meters = 'm', miles = 'mi',
        nautical miles = 'nmi', feet = 'ft', inches = 'in')
        
        
    Returns
    -------
    proba: integer
        the geographical likelihood that the user will visit location new_loc
    
    """
    try:
        # get list of locations visited by the user
        list_locations = set(df_user_locations.loc[user, 'list_locations'])
    except:
        # if the user has visited no location, return 0
        return 0
        
    n = len(list_locations) # number of unique locations visited by the user
    
    # get coordinates pair (lat, lng) for every locations visited by the user 
    coordinates_df = locations_df.set_index('id')[['lat', 'lng']]
    coordinates_pairs = [(coordinates_df['lat'][loc], coordinates_df['lng'][loc])
                         for loc in list_locations]
    
    if type(new_loc)==int:
        # get tuple (lat, lng) for the new location
        new_loc = tuple(locations_df.set_index('id').loc[new_loc, ['lat','lng']].tolist())
    
    # compute distances between the new location and every locations visited by the user 
    new_distances = [haversine(new_loc, loc, unit=unit)
                     for loc in coordinates_pairs]
    
    locations_dist = [haversine(loc1, loc2, unit=unit)
                      for loc1, loc2 in combinations(coordinates_pairs, 2)]
   
    # compute likelihood
    proba = np.mean([density(new_d, locations_dist, n)
                     for new_d in new_distances])
    
    return proba 

In [21]:
%%time
geo_proba(user=20, new_loc=219544)

CPU times: user 117 ms, sys: 13.1 ms, total: 130 ms
Wall time: 75.8 ms


0.0

## 4. Social computations

For every pair of user $(u_i, u_k)$, compute their social similarity score with the Jaccard coefficient as follows:

$$\operatorname{sim}\left(u_{i}, u_{k}\right)=\frac{\left|F\left(u_{i}\right) \cap F\left(u_{k}\right)\right|}{\left|F\left(u_{i}\right) \cup F\left(u_{k}\right)\right|}$$

In [22]:
dict_friends = df_user_friends.set_index('userid')['list_friends'].to_dict()

def social_similarity(user1, user2):
    """Compute Jaccard coefficient to obtain social similarity between two users
    
    Parameters
    ----------
    user1, user2: double
        id of the users
        
    Returns
    -------
    sim: integer
        social similarity between user1 and user2
    
    """
    try:
        num = np.intersect1d(dict_friends[user1], dict_friends[user2]).shape[0]
        den = np.union1d(dict_friends[user1], dict_friends[user2]).shape[0]
        if den==0:
            sim = 0
        else:
            sim = num/den
    except:
        sim = 0
    
    return sim

This score can be exploited into the standard collaborative filtering model:

$$\hat{r}_{i, j}=\frac{\sum_{u_{k} \in F\left(u_{i}\right)} r_{k, j} \cdot \operatorname{Sim}\left(u_{i}, u_{k}\right)}{\sum_{u_{k} \in F\left(u_{i}\right)} \operatorname{Sim}\left(u_{i}, u_{k}\right)}$$

In [23]:
def get_frequence(user, loc):
    """Get the frequence for a user and a location
    
    Parameters
    ----------
    user: double
        user id
        
    loc: double
        location id
    
    Returns
    -------
    freq: integer
        the frequency with which the user visited loc
        0 if he didn't visit the loc
    
    """
    try:
        freq = df_frequencies[(df_frequencies.userid==user) & (df_frequencies.placeid==loc)].freq.values[0]
    except:
        # if the user did not visited the location
        freq = 0
    
    return freq


def prediction(user, loc, list_users=None):
    """Compute the prediction for an user and a location
    
    Parameters
    ----------
    user: double
        user id
        
    loc: double
        location id
        
    list_users: list on integers
        list of users id for which we have some info
        (later, those users will be users in the train set)
        default None
    
    Returns
    -------
    r_hat: integer scoe of collaborative filtering
    
    """
    # list of user's friends
    friends = dict_friends[user].tolist()
    
    if list_users is not None:
        # select only friends that are in list_users
        friends = np.intersect1d(friends, list_users)
    
    num = np.sum([get_frequence(x, loc)*social_similarity(user, x)
                  for x in friends])
    den = np.sum([social_similarity(user, x)
                  for x in friends])
    
    if den!=0:
        r_hat = num/den
    else:
        r_hat = 0
    
    return r_hat

In [24]:
%%time
prediction(user=76, loc=9542)

CPU times: user 163 ms, sys: 41.6 ms, total: 205 ms
Wall time: 115 ms


0.36171055022503246

- Warning : the social similarity can be equal to 0 if the intersection of the set of friends of 2 users is empty. It can be a problem for the computation of the score.
- Because we can divide by 0 if the social similarity of a user and all his friends is null. (no friends in common).
- if the denominateur is equal to 0, we set to 0 the score 

Transform the prediction $\hat{r}_{i,j}$ into a probability as follows:

$$\hat{p}_{i, j}=\frac{\hat{r}_{i, j}}{\max _{l_{j} \in L \backslash L_{i}}\left\{\hat{r}_{i, j}\right\}}$$

In [25]:
def proba(user, loc, df_user_locations=df_user_locations):
    """Compute the probability for an user and a location
    
    Parameters
    ----------
    user: double
        user id
        
    loc: double
        location id
        
    list_locations: list of integer
        list of locations id visited by the user
        if None, then taken from df_user_locations.loc[user, 'list_locations']
        default is None
        
    df_user_locations: pandas DataFrame
        dataframe that associates for each user the list of the successive locations he visited
        index: userid
        columns = [list_locations]
    
    
    Returns
    -------
    p_hat: integer
    
    """
    # list of user's friends
    friends = dict_friends[user]
    
    # select only friends that are in the df_user_locations
    list_users = list(df_user_locations.index)
    friends = np.intersect1d(friends, list_users)
    
    # set of all the locations visited by user's frieds
    all_loc = set(df_user_locations.loc[friends].list_locations.explode())
    
    try:
        # list of locations visited by the user
        user_loc = set(df_user_locations.loc[user, 'list_locations'])
    except:
        # the user is not in df_user_locations
        user_loc = []
    
    # location the user did not visit 
    possible_loc = all_loc.difference(user_loc)
    
    den = np.max([prediction(user, l, list_users) for l in possible_loc])
    #den = np.max(list(map(lambda l: prediction(user, l, list_users), possible_loc)))
    
    if den != 0 :
        p_hat = prediction(user,loc)/den
    else:
        p_hat = 0
    
    return p_hat

- We have a problem of scalability with computing the probability, because the set $L \setminus L_i$ is very big (more than 2 millions location).
- But $\hat{r}_{i, j}$ is actually null if none of the friends of user i visited location j.
- So we can restrain L to the set of locations the friends of i visited.

In [26]:
%%time
proba(user=76, loc=9542)

CPU times: user 17 s, sys: 630 ms, total: 17.6 s
Wall time: 17.6 s


0.2705840302044074

## 5. Generate & Test Recommendations

Compute a recommandation score as follow:

$$\hat{s}_{i, j}=\frac{\hat{p}_{i, j}+p\left(l_{i} | L_{u}\right)}{2}$$

This score given in equation 5 is the average of the collaborative score $\hat{p}_{i, j}$ and the geographic probability $p\left(l_{i} | L_{u}\right)$. It gives a probability user i will like location j.

In [27]:
def score(user, loc, df_user_locations=df_user_locations, method='sum'):
    """Generate a recommendation score for a given user and a location.
    
    Parameters
    ----------
    user: integer
        user id
        
    loc: integer
        location id
        
    df_user_locations: pandas DataFrame
        dataframe that associates for each user the list of the successive locations he visited
        index: userid
        columns = [list_locations]
        
    method: string
        the method used to compute the score ('sum', 'product')
        default is 'sum'
    
    
    Returns
    -------
    score: integer
        the recommandation score
    
    """
    
    p_hat = proba(user=user,
                  loc=loc,
                  df_user_locations=df_user_locations)

    p = geo_proba(user=user,
                  new_loc=loc,
                  df_user_locations=df_user_locations)
    
    if method=='sum':
        score = (p_hat + p)/2
    elif method=='product':
        score = p_hat * p
    else:
        raise ValueError('Unkown value for parameter method.')
    
    return score

In [28]:
%%time
score(user=76, loc=9542)

CPU times: user 17.1 s, sys: 378 ms, total: 17.4 s
Wall time: 17.3 s


0.13711081206075498

Now we wish to determine a list of locations that a given user is prone to like.
To do so, given an user id, we return a list of $n$ locations with the best scores. 

In [29]:
def best_locations(user, n_locations=10, possible_loc='friends',
                   df_user_locations=df_user_locations):
    """Generate the n best location for given user
    among the ones he did not visited yet.
    
    Parameters
    ----------
    user: integer
      user id 
    
    n_locations: integer
      number of recommendations to return
      default is 10
    
    possible_loc: string
      either 'all' or 'friends'
      if 'all', searches among all locations in locations_df
      if 'friends', searches among all locations friends_visited
      default is 'friends'
      
    df_user_locations: pandas DataFrame
        dataframe that associates for each user the list of the successive locations he visited
        index: userid
        columns = [list_locations]
    
    
    Returns 
    -------
    list_scores: list of tuples (location id, score)
        orderer from largest score to smallest score
    
    """
    # list of tuples (location, score)
    list_scores = [] 
    
    try:
        user_loc = set(df_user_locations.loc[user, 'list_locations'])
    except:
        user_loc = []
    
    if possible_loc=='all':
        # set of possible locations
        possible_loc_set = set(locations_df.id).difference(user_loc)
    elif possible_loc=='friends':
        # select only user's friends that are in the df_user_locations
        friends = np.intersect1d(dict_friends[user], list(df_user_locations.index))
        # set of locations visited by user's friends
        possible_loc_set = set(df_user_locations.loc[friends].list_locations.explode())
    else:
        raise ValueError('Unkown value for parameter possible_loc.')
        
    print('number of possible locations:', len(possible_loc_set))
    
    for loc in tqdm(possible_loc_set):
        s = score(user=user,
                  loc=int(loc),
                  df_user_locations=df_user_locations)
        tup = (loc, s)
        list_scores.append(tup)
        
    # sort list of scores from biggest to smallest score
    list_scores.sort(key = lambda x: x[1], reverse=True)
    
    return list_scores[:n_locations]
    

In [30]:
%%time

user = 20
n_locations = 10
best_loc = best_locations(user=user, n_locations=n_locations)
print("User %d should visit location %d" %(user, best_loc[0][0]))
print("All %d best locations for user %d:\n" %(n_locations, user), best_loc)

  0%|          | 0/23 [00:00<?, ?it/s]

number of possible locations: 23


100%|██████████| 23/23 [00:18<00:00,  1.23it/s]

User 20 should visit location 348429
All 10 best locations for user 20:
 [(348429, 0.002055014476423036), (383982, 0.0017065745354810874), (206730, 0.0016279814675760926), (319729, 0.0014922199125443354), (26372, 0.0014728996770382997), (38629, 0.0013866008058440037), (94504, 0.0012541430906971717), (91672, 0.0007498056197647734), (85024, 0.0007408384578650412), (49309, 0.0007345378936888794)]
CPU times: user 21.8 s, sys: 632 ms, total: 22.4 s
Wall time: 18.8 s


### Surprise framework

Documentation: https://surprise.readthedocs.io/en/stable/index.html

Load df_frequencies into the Surprise framework with the load_from_df() function.

In [31]:
# Range of values of frequencies 
reader = Reader(rating_scale=(0, 10)) 

# The columns of df_frequencies must correspond to
# the user id, the place id and frequencies, in that order.
data = Dataset.load_from_df(df_frequencies, reader=reader)

Use the train_test_split function to divide df_frequencies into a 75% train set and a 25% test set.

In [32]:
%%time

trainset, testset = train_test_split(data=data,
                                     test_size=.25,
                                     random_state=42, # seed
                                     shuffle=True)

CPU times: user 3.98 s, sys: 277 ms, total: 4.25 s
Wall time: 4.35 s


In [33]:
trainset.all_users()

range(0, 81003)

Problem: the function train_test_split reinitialized userid to start from 0, as explained in the documentation (https://surprise.readthedocs.io/en/stable/FAQ.html#raw-inner-note)

In [34]:
# retrieve initial id (raw id) of new id (inner id) 0
trainset.to_raw_uid(0)

2080245

Create a GSLR class (i.e. Geographical Social Location Recommendation) and populate a fit() and a estimate() functions.

When we evaluate our model with the test() method that uses the estimate() function, our aim is to estimate the rate given by the user for each location he visited, only using the informations that we have in the train set (we proceed as if all the data in the test set were non-existent).
Hence, for a given user, in order to compute a score for a given location, we can only compute the score based on the locations he visited that are in the train set and on his friends that are in the train set.

In [35]:
class GSLR(AlgoBase):
    def __init__(self): 
        AlgoBase.__init__(self)
        

    def fit(self, trainset):
        AlgoBase.fit(self, trainset)

        dict_locations = defaultdict(list)
        
        for (u,i,_) in trainset.all_ratings():
            # retrieve raw id
            user = self.trainset.to_raw_uid(u)
            loc = self.trainset.to_raw_iid(i)
            # append location to the user's list
            dict_locations[user].append(loc)

        dict_locations = dict(dict_locations) # contains the users locations in the trainset

        # recompute df_users_locations restricted to the train set
        train_data = {'userid': list(dict_locations.keys()),
                      'list_locations': list(dict_locations.values())}
        self.df_user_locations_train = pd.DataFrame(data=train_data).set_index('userid')
        
        return self


    def estimate(self, u, i):
        # retrieve raw id for user and location
        try:
            # user is also in trainset
            user = self.trainset.to_raw_uid(u)
        except:
            # user is not in trainset
            user = int(re.findall(r'\d+', u)[0]) #int(u.replace('UKN__',''))
            
        try:
            # location is also in trainset
            loc = self.trainset.to_raw_iid(i)
        except:
            # location is not in trainset
            loc = int(re.findall(r'\d+', i)[0]) #int(i.replace('UKN__',''))
            
        # compute score
        s = score(user=user,
                  loc=int(loc),
                  df_user_locations=self.df_user_locations_train,
                  method='sum')
        
        return s

In [36]:
algo = GSLR().fit(trainset)

In [37]:
%%time

# number of data to keep in testset
n_test = 20
# reduce the testset: randomly pick n_test observations from testset
testset_reduced = random.choices(testset, k=n_test)
# make predictions
predictions = algo.fit(trainset).test(testset_reduced, verbose=True)

/Users/cedricallain/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:217: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/Users/cedricallain/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:186: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/Users/cedricallain/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:209: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


user: 2267230    item: 3766604    r_ui = 1.24   est = 10.00   {'was_impossible': False}
user: 454732     item: 978531     r_ui = 1.24   est = 0.00   {'was_impossible': False}
user: 56547      item: 93066      r_ui = 0.00   est = 0.00   {'was_impossible': False}
user: 276330     item: 171535     r_ui = 0.00   est = 0.00   {'was_impossible': False}
user: 1027964    item: 110444     r_ui = 0.00   est = 0.00   {'was_impossible': False}
user: 260300     item: 124349     r_ui = 0.00   est = 0.03   {'was_impossible': False}
user: 2168073    item: 300003     r_ui = 0.00   est = 0.03   {'was_impossible': False}
user: 2116191    item: 52298      r_ui = 1.24   est = 0.34   {'was_impossible': False}
user: 1945926    item: 501739     r_ui = 0.00   est = 0.25   {'was_impossible': False}
user: 2134209    item: 5729168    r_ui = 0.00   est = 0.00   {'was_impossible': False}
user: 1035711    item: 79467      r_ui = 0.00   est = 0.06   {'was_impossible': False}
user: 2638123    item: 214191     r_ui = 0

### Precision and recall

We now implement a precision/recall function.
We reproduced this function: https://github.com/NicolasHug/Surprise/blob/master/examples/precision_recall_at_k.py

In [38]:
def precision_recall_at_k(predictions, k=10, threshold=0):
    """Return precision and recall at k metrics for each user.
    
    Parameters
    ----------
    predictions: list of predictions
        [(uid, iid, r_ui, est, details), ...]
    
    k: integer
        number of metrics
        
    threshold: integer
    
    Returns
    -------
    precisions, recalls: integers
    
    """

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r > threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est > threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r > threshold) and (est > threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1

        # Recall@K: Proportion of relevant items that are recommended
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 1

    return precisions, recalls

In [68]:
def compute_means(precisions, recalls):
    """ Compute average precision, average recall and average F1 score
    
    Parameters
    ----------
    precisions, recalls: dictionaries {user id: precision or recall}
        outputs of function precision_recall_at_k() 
    
    
    Returns
    -------
    avg_prec, avg_rec, avg_F1
    
    """
    precisions = list(precisions.values())
    recalls = list(recalls.values())
    
    avg_prec, avg_rec, avg_F1 = np.mean([np.array([prec, rec, 2*prec*rec/(prec+rec+1e-16)])
                                         for (prec, rec) in zip(precisions, recalls)],
                                        axis=0)
    return avg_prec, avg_rec, avg_F1

In [69]:
%%time

# compute precision and recall for all users in test set
precisions, recalls = precision_recall_at_k(predictions, k=10, threshold=0)

# compute average precision, average recall and average F1 score
avg_prec ,avg_rec, avg_F1 = compute_means(precisions, recalls)
print('average precision: %.2f \naverage recall: %.2f \nF1 score: %2f' %(avg_prec, avg_rec, avg_F1))

average precision: 0.20 
average recall: 1.00 
F1 score: 0.200000
CPU times: user 11 ms, sys: 2.58 ms, total: 13.5 ms
Wall time: 16.5 ms


### Cross validation

In [41]:
%%time

kf = KFold(n_splits=3)

algo = GSLR()

c = 0
for trainset, testset in kf.split(data):
    c += 1
    
    # train and test algorithm.
    algo = algo.fit(trainset)
    
    # reduce the testset: randomly pick n_test observations from testset
    testset_reduced = random.choices(testset, k=n_test)
    predictions = algo.test(testset_reduced, verbose=False)
    
    # compute precisions and recalls
    precisions, recalls = precision_recall_at_k(predictions)
    
    # Compute and print average recall precision and F1 score
    avg_prec ,avg_rec, avg_F1 = compute_means(precisions, recalls)
    
    print('Fold %d - precision: %.2f - recall: %.2f - F1: %2f' %(c, avg_prec, avg_rec, avg_F1))


Fold  1
Fold 1 - precision: 0.50 - recall: 0.90 - F1: 0.400000
Fold  2
Fold 2 - precision: 0.10 - recall: 1.00 - F1: 0.100000
Fold  3
Fold 3 - precision: 0.20 - recall: 1.00 - F1: 0.200000
CPU times: user 2min 33s, sys: 7.63 s, total: 2min 40s
Wall time: 2min 45s


## Standard baselines

In [42]:
algo = GSLR().fit(trainset)

### Default prediction
Method: take the mean of all rates in the train set

In [43]:
default_pred = algo.default_prediction()
print("Default prediction:", default_pred)

Default prediction: 0.37832665373237223


In [53]:
# create a vector of "predictions" by adding the default prediction value to every tuple of test set
default_pred_list = [(*x, default_pred, {'was_impossible': False}) for x in testset]

In [67]:
%%time

# compute average precision, average recall and F1 score
precisions, recalls = precision_recall_at_k(default_pred_list, k=10, threshold=0)

avg_prec ,avg_rec, avg_F1 = compute_means(precisions, recalls)

print('average precision: %.2f \naverage recall: %.2f \nF1 score: %2f' %(avg_prec, avg_rec, avg_F1))

average precision: 0.18 
average recall: 0.98 
F1 score: 0.237900


### KNN algorithm

Method: for a given user and a given location, identify the $k$ nearest neighbours of the user (defined as the users, among his friends, that have the biggest social similarity), compute the average rating of the location given by the neighbours, and use the average rating to predict the rating that the user will give to the location.

In [125]:
class KNN(AlgoBase):
    def __init__(self, k): 
        AlgoBase.__init__(self)
        self.k = k
        

    def fit(self, trainset):
        AlgoBase.fit(self, trainset)

        # recompute checkins_df restricted to the train set
        checkins_df_train = {'userid': [], 'placeid': []}
        
        for (u,i,_) in trainset.all_ratings():
            # retrieve raw id
            user = self.trainset.to_raw_uid(u)
            loc = self.trainset.to_raw_iid(i)
            # append user and location in checkins_df_train
            checkins_df_train['userid'].append(user)
            checkins_df_train['placeid'].append(loc)
        
        self.checkins_df_train = pd.DataFrame(data=checkins_df_train)
        
        # list of unique users in the train set
        self.train_users = set(checkins_df_train['userid'])
        
        # recompute df_frequencies restricted to the train set
        df_frequencies_train = self.checkins_df_train.groupby(['userid', 'placeid']).agg({'placeid': 'count'})
        df_frequencies_train.columns = ['freq']
        df_frequencies_train.reset_index(inplace=True)
        df_frequencies_train.freq = normalize_serie(df_frequencies_train.freq)
        
        self.df_frequencies_train = df_frequencies_train
        
        return self
    
    
    def find_knn(self, user):
        """Find the k nearest neighbours of user
        
        The k nearest neighbours of the user are defined as the users,
        among his friends, that have the biggest social similarity with him.
        
        Parameters
        ----------
        user: integer
            user id
            
        Returns
        -------
        knn: list of integers
            list of users id (the knn of input user)
        
        """
        
        friends = dict_friends[user].tolist()
        # restriction to the friends in train set
        friends = np.intersect1d(friends, list(self.train_users))
        
        n_friends = len(friends)
        
        if n_friends==0:
            return None
        elif n_friends <= self.k:
            return friends
        
        # compute social similarity 
        simi_list = [(f, social_similarity(user, f)) for f in friends]
        # sort his friends by social similarity
        simi_list.sort(key = lambda x: x[1], reverse=True)
        # get the k nearest friends of user
        knn = simi_list[:self.k]
        
        return knn
    
    
    def knn_score(self, user, loc):
        """Compute knn score for a user.
        
        The knn score is defined as the average rating of the location
        given by the k neighbours of the user.
        
        Parameters
        ----------
        user: integer
            user id
            
        loc: interger
            location id
            
        
        Return
        ------
        score: integer
        
        """
        # get neighbours of the users
        nn = self.find_knn(user)
        
        if nn is None:
            return 0
        
        df_frequencies_nn = self.df_frequencies_train.copy()
        # filter on user's friends
        df_frequencies_nn = df_frequencies_nn[df_frequencies_nn.userid.isin(nn)]
        # filter on location
        df_frequencies_nn = df_frequencies_nn[df_frequencies_nn.placeid == loc]
        
        # if none of user's friend visited the location, return 0
        if df_frequencies_nn.shape[0]==0:
            score = 0
        else:
            score = np.mean(df_frequencies_nn.freq)
        
        return score


    def estimate(self, u, i):
        # retrieve raw id for user and location
        try:
            # user is also in trainset
            user = self.trainset.to_raw_uid(u)
        except:
            # user is not in trainset
            user = int(re.findall(r'\d+', u)[0]) #int(u.replace('UKN__',''))
            
        try:
            # location is also in trainset
            loc = self.trainset.to_raw_iid(i)
        except:
            # location is not in trainset
            loc = int(re.findall(r'\d+', i)[0]) #int(i.replace('UKN__',''))
            
        # compute score
        s = self.knn_score(user=user, loc=int(loc))
        
        return s

In [126]:
# KNN with 5 neighbours
algo = KNN(5).fit(trainset)

In [129]:
%%time

# reduce the testset: randomly pick n_test observations from testset
testset_reduced = random.choices(testset, k=n_test)

# make predictions
predictions = algo.fit(trainset).test(testset_reduced, verbose=True)

# compute precision and recall for all users in test set
precisions, recalls = precision_recall_at_k(predictions, k=10, threshold=0)

# compute average precision, average recall and average F1 score
avg_prec ,avg_rec, avg_F1 = compute_means(precisions, recalls)
print('average precision: %.2f \naverage recall: %.2f \nF1 score: %2f' %(avg_prec, avg_rec, avg_F1))

user: 663122     item: 765162     r_ui = 0.00   est = 0.00   {'was_impossible': False}
user: 2134894    item: 103109     r_ui = 0.00   est = 0.00   {'was_impossible': False}
user: 265642     item: 298845     r_ui = 0.00   est = 0.00   {'was_impossible': False}
user: 337431     item: 567547     r_ui = 0.00   est = 0.00   {'was_impossible': False}
user: 104320     item: 31784      r_ui = 0.00   est = 0.00   {'was_impossible': False}
user: 360408     item: 750204     r_ui = 1.24   est = 0.00   {'was_impossible': False}
user: 67383      item: 830557     r_ui = 0.00   est = 0.00   {'was_impossible': False}
user: 2233037    item: 666449     r_ui = 1.24   est = 0.00   {'was_impossible': False}
user: 2124639    item: 294215     r_ui = 0.00   est = 0.00   {'was_impossible': False}
user: 223661     item: 52200      r_ui = 0.00   est = 0.00   {'was_impossible': False}
user: 2173859    item: 2243905    r_ui = 0.00   est = 0.00   {'was_impossible': False}
user: 2140934    item: 26134      r_ui = 1.

In [130]:
algo.df_frequencies_train.freq.max()

0.0

Remark: algo.df_frequencies_train is full of 0 for frequency column. That exmplains the better score of KNN.